In [1]:
!pip install transformers
!pip install SentencePiece

     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     ----------------- --------------------- 61.4/134.8 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 134.8/134.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.8 MB 1.7 MB/s eta 0:00:06
    --------------------------------------- 0.1/8.8 MB 1.4 MB/s eta 0:00:07
    --------------------------------------- 0.2/8.8 MB 1.8 MB/s eta 0:00:05
   - -------------------------------------- 0.3/8.8 MB 1.4 MB/s eta 0:00:06
   - -------------------------------------- 0.4/8.8 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.4/8.8 MB 1.6 MB/s eta 0:00:06
   -- ------------------------------------- 0.5/8.8 MB 1.4 MB/s eta 0:00:07
   -- ------------------------------------- 0.5/8.8 MB 1.4 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/8.8 MB 1.5 MB/s eta 0:00:06
   --- -------


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   - -------------------------------------- 41.0/991.5 kB ? eta -:--:--
   -- ------------------------------------- 71.7/991.5 kB 1.3 MB/s eta 0:00:01
   ---- --------------------------------- 112.6/991.5 kB 939.4 kB/s eta 0:00:01
   ----- -------------------------------- 143.4/991.5 kB 853.3 kB/s eta 0:00:01
   ------- ------------------------------ 204.8/991.5 kB 958.4 kB/s eta 0:00:01
   ----------- ---------------------------- 276.5/991.5 kB 1.1 MB/s eta 0:00:01
   -------------- ------------------------- 358.4/991.5 kB 1.2 MB/s eta 0:00:01
   ---------------- ----------------------- 409.6/991.5 kB 1.2 MB/s eta 0:00:01
   ------------------- -------------------- 491.5/991.5 kB 1.3 MB/s eta 0:00:01
   ---------------------- ----------------- 563.2/991.5 kB 1.4 MB/s eta 0:00:01
   -------------------------- ------------- 645.1/991.5 kB 1.4 MB/s eta 0:00:01
   ------------------------------ --------- 757.8/991.5 kB


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

#input_text = "translate English to German: How old are you?"
#input_ids = tokenizer(input_text, return_tensors="pt").input_ids

#outputs = model.generate(input_ids)
#print(tokenizer.decode(outputs[0]))

/apps/jupyter/6.5.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
#!pip install pandas
import pandas as pd

In [24]:
# df = pd.read_json('/content/drive/MyDrive/Information_retrieval/Datasets/train_Input_UserBased_PNC.json')
FILE_PATH = "Datasets/validation_Input_UserBased_PNC.json"
df = pd.read_json(FILE_PATH)
#articles = pd.DataFrame(df['input'].apply(lambda x: x.split("article:", 1)[-1].strip() if "article:" in x else None))

In [3]:
df["input"].head(1)

0    Which category does this article relate to amo...
Name: input, dtype: object

In [ ]:
import json

#candidate_labels = ['women', 'religion', 'politics', 'style & beauty', 'entertainment', 'culture & arts', 'sports', 'science & technology', 'travel', 'business', 'crime', 'education', 'healthy living', 'parents', 'food & drink']
pred_list = []

for index,row in df.iterrows():
    print("Predicting for id: ", row["id"])
    input_text = row["input"]
    #print(input_text)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    
    outputs = model.generate(input_ids)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #print(type(answer))
    #print(answer)
    # Find the label with the highest score
    #max_label = max(zip(answer['labels'], answer['scores']), key=lambda x: x[1])[0]
    
    data = {}
    data["id"] = str(row["id"])
    data["output"] = answer
    pred_list.append(data)

final_json = { "task" : "LaMP_2", "golds" : pred_list }
print(len(pred_list), " items added in the data_list")



In [26]:
# Save the JSON data to a file in the local directory
json_dumped_file = json.dumps(final_json, indent=2)

json_file_path = "baseline_validation_Input_UserBased_PNC.json"
with open(json_file_path, "w") as json_file:
    json_file.write(json_dumped_file) 

In [16]:
sequence_to_classify = "It's hard to find a restaurant that doesn't now place a little card at your table inquiring if the establishment was: (a) really awful; (b) tolerable; (c) sublime."
candidate_labels = ['women', 'religion', 'politics', 'style & beauty', 'entertainment', 'culture & arts', 'sports', 'science & technology', 'travel', 'business', 'crime', 'education', 'healthy living', 'parents', 'food & drink']
answer = classifier(sequence_to_classify, candidate_labels)
#{'labels': ['travel', 'dancing', 'cooking'],
# 'scores': [0.9938651323318481, 0.0032737774308770895, 0.002861034357920289],
# 'sequence': 'one day I will see the world'}

# Find the label with the highest score
max_label = max(zip(answer['labels'], answer['scores']), key=lambda x: x[1])[0]

print(max_label)


business


In [19]:
# len(pred_list)
# string_json_data = json.dumps(final_json, indent=2)
# # Save the JSON data to a file in the local directory
# json_file_path = "validation_Output_UserBased_PNC.json"
# with open(json_file_path, "w") as json_file:
#     json_file.write(string_json_data)

1052

In [ ]:
# Convert the dictionary to JSON format
#json_data = json.dumps(data_list, indent=2)

# Save the JSON data to a file in the local directory
json_file_path = "personalized_train_Input_UserBased_PNC.json"
with open(json_file_path, "w") as json_file:
    json_file.write(json_data)